In [19]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from dtaidistance import dtw
import itertools
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.svm import OneClassSVM
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

import os

import random

from SVM_models.models import evaluate_binary_SVM_standard, evaluate_binary_SVM_custom

# import GunPoint dataset from GunPoint/GunPoint_test.txt and GunPoint/GunPoint_train.txt


def load_dataset(file_path):
    with open(file_path) as f:
        lines = f.readlines()
    # Parse each line, separating the label and features
    labels = []
    features = []
    for line in lines:
        values = list(map(float, line.strip().split()))
        labels.append(int(values[0])-1)  # First value is the label, cast to int
        features.append(values[1:])    # Remaining values are features
    # Convert lists to numpy arrays for easier handling
    labels = np.array(labels)
    features = np.array(features)
    # normalize dataset to 0 1
    features = (features - features.min()) / (features.max() - features.min())
    return labels, features

# Load the GunPoint dataset

#dataset_path with join

train_path = os.path.join("GunPoint", "GunPoint_TRAIN.txt")
test_path = os.path.join("GunPoint", "GunPoint_TEST.txt")

train_labels, train_features = load_dataset(train_path)
INITIAL_TEST_LABELS, INITIAL_TEST_FEATURES = load_dataset(test_path)

# split test in validation (50) and test (100)



In [20]:
EXPS = [
    "RANDOM",
    "RANDOM_BEST",
    "BAYESIAN_HAMILTONIAN",
    "BAYESIAN_MODEL",
]

OPT_TIMES = 30

CONFIG = {
    "N_QUBITS": 2,
    "SPARSITY": 0.95,
    "ENCODING": "euler",
}

SEEDS = [1, 2, 3, 4, 5, 6, 7, 8, 9]
EXP = "BAYESIAN_HAMILTONIAN"


In [21]:
import os
def save_results(EXP, encoding, seed, train_accuracy, val_accuracy, test_accuracy, Hamiltonian_c, hidden_alphas):
    # Create folder structure
    folder_name = f"{EXP}__{encoding}"
    seed_folder = os.path.join("exps_results", folder_name, f"exp_{seed}")
    os.makedirs(seed_folder, exist_ok=True)

    # Save configuration and OPT_TIMES to a text file
    with open(os.path.join(seed_folder, "config.txt"), "w") as f:
        f.write(f"Experiment: {EXP}\n")
        f.write(f"Encoding: {encoding}\n")
        f.write(f"Number of Qubits: {CONFIG['N_QUBITS']}\n")
        f.write(f"Sparsity Coefficient: {CONFIG['SPARSITY']}\n")
        f.write(f"OPT_TIMES: {OPT_TIMES}\n")

    # Save train, validation, and test accuracy
    with open(os.path.join(seed_folder, "train_accuracy.txt"), "w") as f:
        f.write(f"Train Accuracy: {train_accuracy}\n")
    
    with open(os.path.join(seed_folder, "val_accuracy.txt"), "w") as f:
        f.write(f"Validation Accuracy: {val_accuracy}\n")

    with open(os.path.join(seed_folder, "test_accuracy.txt"), "w") as f:
        f.write(f"Test Accuracy: {test_accuracy}\n")

    # Save Hamiltonian_c and hidden_alphas
    with open(os.path.join(seed_folder, "Hamiltonian_c.txt"), "w") as f:
        f.write(str(Hamiltonian_c))

    with open(os.path.join(seed_folder, "hidden_alphas.txt"), "w") as f:
        f.write(str(hidden_alphas))


In [22]:
from SVM_models.QuantumSVM import QuantumSVM
from sklearn.metrics import accuracy_score

In [23]:
from bayes_opt import BayesianOptimization

I = np.array([[1, 0], [0, 1]])  # Identity
X = np.array([[0, 1], [1, 0]])  # Pauli-X
Y = np.array([[0, -1j], [1j, 0]])  # Pauli-Y
Z = np.array([[1, 0], [0, -1]]) 

L = 150

def generate_H_from_C(coefficients):
    """Generates a random Hamiltonian for the given number of qubits by combining Pauli matrices."""
    pauli_matrices = [I, X, Y, Z]  # Random coefficients for each term
    H = None
    # Generate all possible combinations of Pauli matrices for the given number of qubits
    for i in range(4**CONFIG["N_QUBITS"]):
        indices = np.unravel_index(i, (4,) * CONFIG["N_QUBITS"])
        kron_product = pauli_matrices[indices[0]]
        for idx in indices[1:]:
            kron_product = np.kron(kron_product, pauli_matrices[idx])
        kron_product = np.array(kron_product, dtype=complex)
        if H is None:
            H = coefficients[i] * kron_product
        else:
            H += coefficients[i] * kron_product
    H = H / np.linalg.norm(H)  # Normalize the Hamiltonian
    return H

def generate_random_alphas():
    return np.random.rand(L)

In [24]:
from time import sleep

for SEED in SEEDS:

    random.seed(SEED)

    test_labels, val_labels, test_features, val_features = train_test_split(INITIAL_TEST_LABELS, INITIAL_TEST_FEATURES, test_size=0.33, random_state=SEED)

    fixed_alphas = generate_random_alphas()

    def objective_function(**kwargs):
        # Convert kwargs to a coefficient array
        coefficients = np.array([kwargs[f"coeff_{i}"] for i in range(4**CONFIG["N_QUBITS"])])
        #print("Coefficients:", coefficients)
        # Fit the QuantumSVM model with the generated Hamiltonian
        quantum_svm = QuantumSVM(n_qubits=CONFIG["N_QUBITS"], Hamiltonian_c=coefficients, alphas_c=fixed_alphas, embedding_code=CONFIG["ENCODING"], sparsity_coefficient=CONFIG["SPARSITY"])
        quantum_svm.fit(train_features, train_labels)
        
        y_val = quantum_svm.predict(val_features)
        val_accuracy = accuracy_score(val_labels, y_val)
        return val_accuracy


    pbounds = {f"coeff_{i}": (0, 1) for i in range(4**CONFIG["N_QUBITS"])}

    # Initialize Bayesian Optimizer
    optimizer = BayesianOptimization(
        f=objective_function,
        pbounds=pbounds,
        random_state=42,
        verbose=2
    )

    optimizer.maximize(
        init_points=5,  # Number of random initial points
        n_iter=OPT_TIMES-5        # Number of optimization steps
    )
    # Retrieve the best coefficients
    best_coefficients = [optimizer.max["params"][f"coeff_{i}"] for i in range(4**CONFIG["N_QUBITS"])]

    print("Best Coefficients:", best_coefficients)
    print("Best Val Accuracy:", optimizer.max["target"])

    quantum_svm = QuantumSVM(n_qubits=CONFIG["N_QUBITS"], Hamiltonian_c=best_coefficients, alphas_c=fixed_alphas, embedding_code=CONFIG["ENCODING"], sparsity_coefficient=CONFIG["SPARSITY"])

    quantum_svm.fit(train_features, train_labels)
    y_train = quantum_svm.predict_on_train()
    train_accuracy = accuracy_score(train_labels, y_train)
    print(f"Train Accuracy: {train_accuracy}")

    y_val = quantum_svm.predict(val_features)
    val_accuracy = accuracy_score(val_labels, y_val)
    print(f"Validation Accuracy: {val_accuracy}")

    y_test = quantum_svm.predict(test_features)
    accuracy = accuracy_score(test_labels, y_test)
    print(f"Test Accuracy: {accuracy}")


    # now I want to save the model Hamiltonian_c and hidden_alphas
    print("Saving model...")
    Hamiltonian_c = quantum_svm.Hamiltonian_c
    hidden_alphas = quantum_svm.hidden_alphas

    #print("Hamiltonian_c: ", Hamiltonian_c)
    #print("hidden_alphas: ", hidden_alphas)
    save_results(EXP, CONFIG["ENCODING"], SEED, train_accuracy, val_accuracy, accuracy, Hamiltonian_c, hidden_alphas)
    sleep(60)

|   iter    |  target   |  coeff_0  |  coeff_1  | coeff_10  | coeff_11  | coeff_12  | coeff_13  | coeff_14  | coeff_15  |  coeff_2  |  coeff_3  |  coeff_4  |  coeff_5  |  coeff_6  |  coeff_7  |  coeff_8  |  coeff_9  |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Non-zero alphas: 5


Building TRAIN_KERNEL:   0%|          | 0/1225 [00:00<?, ?it/s]

Building TEST_KERNEL: 100%|██████████| 2500/2500 [01:06<00:00, 37.39it/s]


| 1         | 0.48      | 0.3745    | 0.9507    | 0.732     | 0.5987    | 0.156     | 0.156     | 0.05808   | 0.8662    | 0.6011    | 0.7081    | 0.02058   | 0.9699    | 0.8324    | 0.2123    | 0.1818    | 0.1834    |
Non-zero alphas: 5


Building TEST_KERNEL: 100%|██████████| 2500/2500 [01:10<00:00, 35.69it/s]


...

Train Accuracy: 0.84


Building TEST_KERNEL: 100%|██████████| 2500/2500 [01:02<00:00, 39.94it/s]


Validation Accuracy: 0.66


Building TEST_KERNEL: 100%|██████████| 5000/5000 [02:05<00:00, 39.82it/s]


Test Accuracy: 0.64
Saving model...
